In [11]:
%pip install pyspark
%pip install pandas
%pip install openpyxl


[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
import fitz

In [13]:
spark = SparkSession.builder \
    .appName("Read PDF and Word Count") \
    .getOrCreate()

In [32]:
textFile1 = spark.read.text("Anies Cak Imin Visi Misi.txt")

In [33]:
textFile1

DataFrame[value: string]

In [34]:
textFile2 = spark.read.text("PRABOWOGIBRAN_VISI_MISI.txt")

In [35]:
textFile2

DataFrame[value: string]

In [36]:
textFile3 = spark.read.text("1242-visi-misi-ganjar-pranowo-mahfud-md.txt")

In [37]:
textFile3

DataFrame[value: string]

In [42]:
# Fungsi untuk menghitung total jumlah kata dalam sebuah file
def totalWordCount(textFile):
    return (
        textFile
        .select(sf.explode(sf.split(sf.lower(textFile.value), r"\s+")).alias("word"))
        .filter(
            (sf.col("word") == "ekonomi") |
            (sf.col("word") == "pendidikan") |
            (sf.col("word") == "kesehatan") |
            (sf.col("word") == "kemandirian") |
            (sf.col("word") == "kesejahteraan") |
            (sf.col("word") == "keberlanjutan") |
            (sf.col("word") == "inovasi") |
            (sf.col("word") == "pemberdayaan") |
            (sf.col("word") == "transparansi") |
            (sf.col("word") == "perubahan")
        )
        .groupBy("word")
        .count()
        .agg(sf.sum("count").alias("total_count"))
    )

# Menghitung total kata dari textFile1, textFile2, dan textFile3
totalWords1 = totalWordCount(textFile1).collect()[0]["total_count"]
totalWords2 = totalWordCount(textFile2).collect()[0]["total_count"]
totalWords3 = totalWordCount(textFile3).collect()[0]["total_count"]

# Menampilkan total kata setiap file
print(f"Total words in file 1: {totalWords1}")
print(f"Total words in file 2: {totalWords2}")
print(f"Total words in file 3: {totalWords3}")

Total words in file 1: 357
Total words in file 2: 206
Total words in file 3: 104


In [38]:
from pyspark.sql import functions as sf

# Fungsi untuk menghitung jumlah kata dari sebuah file
def countWords(textFile):
    return (
        textFile
        .select(sf.explode(sf.split(sf.lower(textFile.value), r"\s+")).alias("word"))
        .filter(
            (sf.col("word") == "ekonomi") |
            (sf.col("word") == "pendidikan") |
            (sf.col("word") == "kesehatan") |
            (sf.col("word") == "kemandirian") |
            (sf.col("word") == "kesejahteraan") |
            (sf.col("word") == "keberlanjutan") |
            (sf.col("word") == "inovasi") |
            (sf.col("word") == "pemberdayaan") |
            (sf.col("word") == "transparansi") |
            (sf.col("word") == "perubahan")
        )
        .groupBy("word")
        .count()
    )

In [39]:
# Menghitung jumlah kata dari textFile1, textFile2, dan textFile3
wordCounts1 = countWords(textFile1).withColumnRenamed("count", "count1")
wordCounts2 = countWords(textFile2).withColumnRenamed("count", "count2")
wordCounts3 = countWords(textFile3).withColumnRenamed("count", "count3")

# Menggabungkan hasil dari ketiga file berdasarkan kolom 'word'
combinedWordCounts = (
    wordCounts1
    .join(wordCounts2, "word", "outer")
    .join(wordCounts3, "word", "outer")
    .na.fill(0)  # Mengisi nilai kosong dengan 0 jika kata tidak ditemukan di file tertentu
)

# Menampilkan hasil
combinedWordCounts.show()

+-------------+------+------+------+
|         word|count1|count2|count3|
+-------------+------+------+------+
|      ekonomi|    96|    91|    45|
|      inovasi|    14|    10|    15|
|keberlanjutan|     5|    12|     0|
|  kemandirian|    12|    12|     3|
|    kesehatan|    73|    28|    16|
|kesejahteraan|    42|    18|    14|
| pemberdayaan|    12|     6|     0|
|   pendidikan|    74|    24|     7|
|    perubahan|    28|     5|     4|
| transparansi|     1|     0|     0|
+-------------+------+------+------+

